In [78]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import imutils
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from tabulate import tabulate

def flatten_image(image):
    return np.array(image).flatten()

def flatten_images(images):
    return np.array([flatten_image(image) for image in images])

image_size = 32

In [79]:
def read_image(class_path, image_file):
    image_path = os.path.join(class_path, image_file)
    image =  cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    return image

class_labels_list = []
images_list = []

data_directory = './dataset/'

#missing O
directories = ['@', "&", "#", "$", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K','L','M','N','P','Q','R','S','T','U','V','W','X','Y','Z','0','1','2','3','4','5','6','7','8','9']

for class_label, class_name in enumerate(directories):
    class_path = os.path.join(data_directory, class_name)
    for image_file in os.listdir(class_path):
        if image_file.endswith('.jpg'):
            image = read_image(class_path, image_file)
            images_list.append(image)
            class_labels_list.append(class_label)

data_x = images_list
data_y = class_labels_list

X_train, X_test, y_train, y_test = train_test_split(data_x,data_y, 
                                                    test_size=0.25, 
                                                    random_state=13)

In [80]:
pca = PCA(n_components=50).fit(flatten_images(X_train))
X_train_pca = pca.transform(flatten_images(X_train))
X_test_pca = pca.transform(flatten_images(X_test))


In [81]:
pca_knn_classifier = KNeighborsClassifier(n_neighbors=5) 
pca_knn_classifier.fit(X_train_pca, y_train)
pca_knn_predictions = pca_knn_classifier.predict(X_test_pca)
pca_knn_accuracy = accuracy_score(y_test, pca_knn_predictions)

print("Random KNN PCA Classifier:")
print(f"Accuracy: {pca_knn_accuracy}")

random_sample = X_test[0]
print(pca.transform([flatten_image(random_sample)]))
random_sample_prediction = pca_knn_classifier.predict(pca.transform([flatten_image(random_sample)]))
print(f"Random Sample Prediction: {directories[random_sample_prediction[0]]}")

Random KNN PCA Classifier:
Accuracy: 0.8488095238095238
[[-742.98917851 -750.37291838 -471.05954583 -920.69324995 -166.42414087
   425.91898266  259.81878944 -581.49243606   60.47920443 -342.50901376
  -126.3295728   271.37756071 -326.41589795  -55.6521697    50.46476267
  -146.14874861  234.93447336  458.17823124 -233.49782018   56.75270276
  -173.39259059  104.16886645 -180.4113004    80.24513824  285.59417581
   -72.29748042 -165.09719963   63.07615361  -47.76110228 -139.6811098
   231.85451967  267.15498381  -16.44162893   61.44502104  105.85120987
    89.76901506  101.68025699   -7.46511645   88.62928699   35.49747986
    14.42323253  -47.20499313  -73.7413769    21.80062142  -11.16598203
    -4.08473544   37.70297711  -95.55293124    3.20361658   69.48087697]]
Random Sample Prediction: V


In [82]:

def apply_sobel(image):
    gx = cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize=5)
    gy = cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize=5)

    gradient = [gx, gy]
    
    return gradient

x_train_sobel = [apply_sobel(image) for image in X_train]
x_test_sobel = [apply_sobel(image) for image in X_test]

In [83]:

x_test_sobel = flatten_images(x_test_sobel)
x_train_sobel = flatten_images(x_train_sobel)

sobel_knn_classifier = KNeighborsClassifier(n_neighbors=5)

sobel_knn_classifier.fit(x_train_sobel, y_train)

sobel_knn_predictions = sobel_knn_classifier.predict(x_test_sobel)

sobel_knn_accuracy = accuracy_score(y_test, sobel_knn_predictions)

print("Random KNN Direction Classifier:")
print(f"Accuracy: {sobel_knn_accuracy}")

Random KNN Direction Classifier:
Accuracy: 0.7642857142857142


In [84]:
X_train_combined = np.concatenate((X_train_pca, x_train_sobel), axis=1)
X_test_combined = np.concatenate((X_test_pca, x_test_sobel), axis=1)


merged_knn_classifier = KNeighborsClassifier(n_neighbors=5)

merged_knn_classifier.fit(X_train_combined, y_train)

merged_knn_predictions = merged_knn_classifier.predict(X_test_combined)

merged_knn_accuracy = accuracy_score(y_test, merged_knn_predictions)

print("Random KNN Combined Classifier:")
print(f"Accuracy: {merged_knn_accuracy}")


Random KNN Combined Classifier:
Accuracy: 0.7642857142857142


In [85]:
model = pca_knn_classifier

def sort_contours(cnts, method="left-to-right"):
    reverse = False
    i = 0
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(
        *sorted(zip(cnts, boundingBoxes), key=lambda b: b[1][i], reverse=reverse)
    )
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)


def get_letters(img):
    letters = []
    image = cv2.imread(img)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, thresh1 = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    dilated = cv2.dilate(thresh1, None, iterations=2)
    cnts = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sort_contours(cnts, method="left-to-right")[0]
    for c in cnts:
        if cv2.contourArea(c) > 10:
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        roi = gray[y : y + h, x : x + w]
        thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        thresh = cv2.resize(thresh, (image_size,image_size))
        thresh = pca.transform([flatten_image(thresh)])
        ypred = model.predict(thresh)
        [x] = ypred
        letters.append(x)
    return letters, image


def get_word(letter):
    word = ''
    for letter in letters:
        word += directories[letter]
    return word
    


letters, image = get_letters(
    "./dataset/words/validation/VALIDATION_0010.jpg"
)

word = get_word(letters)



print(word)
plt.imshow(image)

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [98]:
# Suppose you have the confusion matrix stored in knn_confusion_matrix
knn_pca_confusion_matrix = confusion_matrix(y_test, pca_knn_predictions)

# Get the labels for the confusion matrix for the first 10 categories
labels = pca_knn_classifier.classes_[:10]

# Get the first 10 rows and columns of the confusion matrix
confusion_matrix_10 = knn_pca_confusion_matrix[:10, :10]
row_labels = ["True/Predicted"] + list(labels)

# Create an array with numbers from 1 to 10
numbers = np.arange(1, 11)

# Add an extra row and column for "Predicted/True" labels and numbers
header_labels = ["Predicted/true"] + list(numbers)

# Create a new confusion matrix with an additional row and column
new_confusion_matrix = np.zeros((11, 11), dtype=int)
new_confusion_matrix[1:, 1:] = labels
new_confusion_matrix[1:, 0] = labels

# Convert the confusion matrix to a table
table = tabulate(confusion_matrix_10, headers=header_labels, showindex=labels-3, tablefmt="grid")

# Print the confusion matrix table
print(table)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [88]:
num_categories = knn_pca_confusion_matrix.shape[0]

metrics = {}

for i in range(num_categories):
    TP = knn_pca_confusion_matrix[i, i]
    FN = np.sum(knn_pca_confusion_matrix[i, :]) - TP
    FP = np.sum(knn_pca_confusion_matrix[:, i]) - TP
    TN = np.sum(knn_pca_confusion_matrix) - TP - FN - FP

    sensitivity = TP / (TP + FN)
    specificity = TN / (TN + FP)
    precision = TP / (TP + FP)
    recall = sensitivity
    f1_score = 2 * (precision * recall) / (precision + recall)

    metrics[i] = {"Sensitivity": sensitivity,
                  "Specificity": specificity,
                  "Precision": precision,
                  "Recall": recall,
                  "F1-score": f1_score}
print("KNN-PCA-Sensitivity: ",sensitivity)
print("KNN-PCA-Specificity: ",specificity)
print("KNN-PCA-Precision: ",precision)
print("KNN-PCA-Recall: ",recall)
print("KNN-PCA-F1-score: ",f1_score)

KNN-PCA-Sensitivity:  1.0
KNN-PCA-Specificity:  0.9987730061349693
KNN-PCA-Precision:  0.9615384615384616
KNN-PCA-Recall:  1.0
KNN-PCA-F1-score:  0.9803921568627451


In [99]:
# Suppose you have the confusion matrix stored in knn_confusion_matrix
knn_sobel_confusion_matrix = confusion_matrix(y_test, sobel_knn_predictions)

# Get the labels for the confusion matrix for the first 10 categories
labels = sobel_knn_classifier.classes_[:10]

# Get the first 10 rows and columns of the confusion matrix
confusion_matrix_10 = knn_sobel_confusion_matrix[:10, :10]
row_labels = ["True/Predicted"] + list(labels)

# Create an array with numbers from 1 to 10
numbers = np.arange(1, 11)

# Add an extra row and column for "Predicted/True" labels and numbers
header_labels = ["Predicted/true"] + list(numbers)

# Create a new confusion matrix with an additional row and column
new_confusion_matrix = np.zeros((11, 11))
new_confusion_matrix[1:, 1:] = labels
new_confusion_matrix[1:, 0] = labels

# Convert the confusion matrix to a table
table = tabulate(confusion_matrix_10, headers=header_labels, showindex=labels-3, tablefmt="grid")

# Print the confusion matrix table
print(table)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [90]:
num_categories = knn_sobel_confusion_matrix.shape[0]

metrics = {}

for i in range(num_categories):
    TP = knn_sobel_confusion_matrix[i, i]
    FN = np.sum(knn_sobel_confusion_matrix[i, :]) - TP
    FP = np.sum(knn_sobel_confusion_matrix[:, i]) - TP
    TN = np.sum(knn_sobel_confusion_matrix) - TP - FN - FP

    sensitivity = TP / (TP + FN)
    specificity = TN / (TN + FP)
    precision = TP / (TP + FP)
    recall = sensitivity
    f1_score = 2 * (precision * recall) / (precision + recall)

    metrics[i] = {"Sensitivity": sensitivity,
                  "Specificity": specificity,
                  "Precision": precision,
                  "Recall": recall,
                  "F1-score": f1_score}
print("KNN-SOBEL-Sensitivity",sensitivity)
print("KNN-SOBEL-Specificity",specificity)
print("KNN-SOBEL-Precision",precision)
print("KNN-SOBEL-Recall",recall)
print("KNN-SOBEL-F1-score",f1_score)

KNN-SOBEL-Sensitivity 1.0
KNN-SOBEL-Specificity 0.996319018404908
KNN-SOBEL-Precision 0.8928571428571429
KNN-SOBEL-Recall 1.0
KNN-SOBEL-F1-score 0.9433962264150945


In [91]:
# Suppose you have the confusion matrix stored in svm_confusion_matrix
svm_pca_confusion_matrix = confusion_matrix(y_test, pca_svm_predictions)

# Get the labels for the confusion matrix for the first 10 categories
labels = pca_svm_classifier.classes_[:10]

# Get the first 10 rows and columns of the confusion matrix
confusion_matrix_10 = svm_pca_confusion_matrix[:10, :10]
row_labels = ["True/Predicted"] + list(labels)

# Create an array with numbers from 1 to 10
numbers = np.arange(1, 11)

# Add an extra row and column for "Predicted/True" labels and numbers
header_labels = ["Predicted/true"] + list(numbers)

# Create a new confusion matrix with an additional row and column
new_confusion_matrix = np.zeros((11, 11), dtype=np.int)
new_confusion_matrix[1:, 1:] = labels
new_confusion_matrix[1:, 0] = labels

# Convert the confusion matrix to a table
table = tabulate(confusion_matrix_10, headers=header_labels, showindex=labels-3, tablefmt="grid")

# Print the confusion matrix table
print(table)

ValueError: Found input variables with inconsistent numbers of samples: [840, 1104]

In [ ]:
num_categories = svm_pca_confusion_matrix.shape[0]

metrics = {}

for i in range(num_categories):
    TP = svm_pca_confusion_matrix[i, i]
    FN = np.sum(svm_pca_confusion_matrix[i, :]) - TP
    FP = np.sum(svm_pca_confusion_matrix[:, i]) - TP
    TN = np.sum(svm_pca_confusion_matrix) - TP - FN - FP

    sensitivity = TP / (TP + FN)
    specificity = TN / (TN + FP)
    precision = TP / (TP + FP)
    recall = sensitivity
    f1_score = 2 * (precision * recall) / (precision + recall)

    metrics[i] = {"Sensitivity": sensitivity,
                  "Specificity": specificity,
                  "Precision": precision,
                  "Recall": recall,
                  "F1-score": f1_score}
print("SVM-PCA-Sensitivity: ",sensitivity)
print("SVM-PCA-Specificity: ",specificity)
print("SVM-PCA-Precision: ",precision)
print("SVM-PCA-Recall: ",recall)
print("SVM-PCA-F1-score: ",f1_score)

In [ ]:
# Suppose you have the confusion matrix stored in svm_confusion_matrix
svm_sobel_confusion_matrix = confusion_matrix(y_test, sobel_svm_predictions)

# Get the labels for the confusion matrix for the first 10 categories
labels = sobel_svm_classifier.classes_[:10]

# Get the first 10 rows and columns of the confusion matrix
confusion_matrix_10 = svm_sobel_confusion_matrix[:10, :10]
row_labels = ["True/Predicted"] + list(labels)

# Create an array with numbers from 1 to 10
numbers = np.arange(1, 11)

# Add an extra row and column for "Predicted/True" labels and numbers
header_labels = ["Predicted/true"] + list(numbers)

# Create a new confusion matrix with an additional row and column
new_confusion_matrix = np.zeros((11, 11), dtype=np.int)
new_confusion_matrix[1:, 1:] = labels
new_confusion_matrix[1:, 0] = labels

# Convert the confusion matrix to a table
table = tabulate(confusion_matrix_10, headers=header_labels, showindex=labels-3, tablefmt="grid")

# Print the confusion matrix table
print(table)

In [ ]:
num_categories = svm_sobel_confusion_matrix.shape[0]

metrics = {}

for i in range(num_categories):
    TP = svm_sobel_confusion_matrix[i, i]
    FN = np.sum(svm_sobel_confusion_matrix[i, :]) - TP
    FP = np.sum(svm_sobel_confusion_matrix[:, i]) - TP
    TN = np.sum(svm_sobel_confusion_matrix) - TP - FN - FP

    sensitivity = TP / (TP + FN)
    specificity = TN / (TN + FP)
    precision = TP / (TP + FP)
    recall = sensitivity
    f1_score = 2 * (precision * recall) / (precision + recall)

    metrics[i] = {"Sensitivity": sensitivity,
                  "Specificity": specificity,
                  "Precision": precision,
                  "Recall": recall,
                  "F1-score": f1_score}
print("SVM-SOBEL-Sensitivity: ",sensitivity)
print("SVM-SOBEL-Specificity: ",specificity)
print("SVM-SOBEL-Precision: ",precision)
print("SVM-SOBEL-Recall: ",recall)
print("SVM-SOBEL-F1-score: ",f1_score)